In [2]:
# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator


In [4]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')



Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001, decay=1e-6), metrics=['accuracy'])



In [6]:
# Train the neural network/model
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')

Epoch 1/50
448/448 [==============================] - 233s 521ms/step - loss: 1.7886 - accuracy: 0.2666 - val_loss: 1.6793 - val_accuracy: 0.3504
Epoch 2/50
448/448 [==============================] - 231s 516ms/step - loss: 1.6225 - accuracy: 0.3672 - val_loss: 1.5364 - val_accuracy: 0.4252
Epoch 3/50
448/448 [==============================] - 229s 512ms/step - loss: 1.5246 - accuracy: 0.4129 - val_loss: 1.4536 - val_accuracy: 0.4471
Epoch 4/50
448/448 [==============================] - 219s 489ms/step - loss: 1.4510 - accuracy: 0.4434 - val_loss: 1.3927 - val_accuracy: 0.4729
Epoch 5/50
448/448 [==============================] - 211s 471ms/step - loss: 1.3883 - accuracy: 0.4737 - val_loss: 1.3471 - val_accuracy: 0.4937
Epoch 6/50
448/448 [==============================] - 213s 476ms/step - loss: 1.3359 - accuracy: 0.4910 - val_loss: 1.2975 - val_accuracy: 0.5085
Epoch 7/50
448/448 [==============================] - 227s 505ms/step - loss: 1.2927 - accuracy: 0.5097 - val_loss: 1.2640 -

In [7]:
from keras.models import model_from_json
json_file = open('model/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

emotion_model.load_weights("model/emotion_model.h5")
print("Loaded model from disk")
emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001, decay=1e-6), metrics=['accuracy'])
emotion_model.evaluate(validation_generator)

Loaded model from disk
113/113 [==============================] - 11s 96ms/step - loss: 1.2019 - accuracy: 0.6216


[1.2018555402755737, 0.6216216087341309]